In [ ]:
# Used for debugging
import pdb; 
pdb.set_trace() # define a breakpoint

**Command Description**
list Show the current location in the file

1.   **h**(elp) Show a list of commands, or find help on a specific command
2.   **q**(uit) Quit the debugger and the program
3.   **c**(ontinue) Quit the debugger, continue in the program
4.   **n**(ext) Go to the next step of the program
5.   **Repeat** the previous command
6.   **p**(rint) Print variables
7.   **s**(tep) Step into a subroutine
8.   **r**(eturn) Return out of a subroutine

# UTILS

In [ ]:
import librosa
import librosa.display

def plot_melspectrogram(songname:str):
  y, sr = librosa.load(songname, mono=True, duration=2, offset=2*2)
  ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512, n_mels=64)
  ps = librosa.power_to_db(ps**2)

  D = librosa.stft(y)  # STFT of y
  S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

  plt.figure()
  librosa.display.specshow(S_db)
  plt.colorbar()

In [ ]:
# Utilities to run evaluation of classifiers


from sklearn.model_selection import KFold, train_test_split 
from sklearn import metrics

# We can perform both KFold validation and hold-out validation, but because of computational cost only the latter is used.
def evaluate_classifier(classifier, n_folds, X_train_split, y_train_split, X_validation_split = None, y_validation_split = None, fit = True):
  """
  Utilities to run evaluation of classifiers
  The splits are not performed in these methods because validation sets need to be stored in order to have a reliable evaluation of 
  models that are stored and then reloaded.

  If n_folds is falsy, perform a validation with a single test/validation split, using the other parameters

  """

  if n_folds:
    accuracy, precision, recall, fmacro, fmicro = _evaluate_classifier_with_folds(classifier, fit, n_folds)
  else:
    accuracy, precision, recall, fmacro, fmicro = _evaluate_classifier_with_split(classifier, fit, X_train_split, X_validation_split, y_train_split, y_validation_split)
   
  print("Accuracy: ", accuracy)
  print("Precision: ", precision)
  print("Recall: ", recall)
  print("F1-micro: ", fmicro)
  print("F1-macro: ", fmacro)

  return accuracy, precision, recall, fmacro, fmicro


def _evaluate_classifier_with_folds(classifier, fit, n_folds):

  accuracy, precision, recall, fmacro, fmicro = 0, 0, 0, 0, 0
  X_train_array = np.array(X_train)

  k_fold = KFold(n_splits=n_folds, random_state=0, shuffle=True)
  for idx, (train_index, validation_index) in enumerate(k_fold.split(X_train_array)):
    
    print(f"Fold #{idx}")
    print(f"Fitting the model...")

    if fit:
      classifier.fit(X_train_array[train_index], y_train[train_index])
    
    print(f"Model fitted. Predicting on validation set...")
    predicted = classifier.predict(X_train_array[validation_index])
    accuracy = accuracy + metrics.accuracy_score(y_train[validation_index], predicted)
    precision = precision + metrics.precision_score(y_train[validation_index], predicted, average='micro')
    recall = recall + metrics.recall_score(y_train[validation_index], predicted, average='micro')
    fmicro = fmicro + metrics.f1_score(y_train[validation_index], predicted, average='micro')
    fmacro = fmacro + metrics.f1_score(y_train[validation_index], predicted, average='macro')
  
  accuracy /= n_folds
  precision /= n_folds
  recall /= n_folds
  fmicro /= n_folds
  fmacro /= n_folds
  
  return accuracy, precision, recall, fmacro, fmicro


def _evaluate_classifier_with_split(classifier, fit, X_train_split, X_validation_split, y_train_split, y_validation_split):
  
  if fit:
    print(f"Fitting the model...")
    classifier.fit(X_train_split, y_train_split)
  
  print(f"Model fitted. Prediting on validation set...")
  predicted = classifier.predict(X_validation_split)

  accuracy = metrics.accuracy_score(y_validation_split, predicted)
  precision = metrics.precision_score(y_validation_split, predicted, average='macro')
  recall = metrics.recall_score(y_validation_split, predicted, average='macro')  
  fmicro = metrics.f1_score(y_validation_split, predicted, average='micro')
  fmacro = metrics.f1_score(y_validation_split, predicted, average='macro')

  return accuracy, precision, recall, fmacro, fmicro


In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
    
    fig,axs = plt.subplots(2)
    axs[0].plot(history.history["accuracy"],label="train accuracy")
    axs[0].plot(history.history["val_accuracy"],label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc='lower right')
    axs[0].set_title("Accuracy eval")
    
    axs[1].plot(history.history["loss"],label="train error")
    axs[1].plot(history.history["val_loss"],label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc='upper right')
    plt.show()

# SONG INFO CLASSIFICATION 

## Dataset *Preprocessing*

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

dataset_url = "features_3_sec.csv"
dataset = pd.read_csv(dataset_url, sep = ',')
dataset_X = dataset.drop(['label', 'filename'], axis=1)
dataset_Y =  np.array(dataset.label.astype(int))

X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.33, random_state=0)
X_train_split, X_validation, y_train_split, y_validation = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

In [ ]:
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print("\n")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")
print("\n")
print(f"X_train_split: {X_train_split.shape}")
print(f"y_train_split: {y_train_split.shape}")
print("\n")
print(f"X_validation: {X_validation.shape}")
print(f"y_validation: {y_validation.shape}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# display dataset information

print(f'\n\n Sample of the train set of {len(X_train.index)} elements \n')
display(X_train.sample(n=5))

print(f'\n\n Sample of the test set of {len(X_test.index)} elements \n')
display(X_test.sample(n=5))

def plot_data(data, name):
  labels = ['blues','classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
  data = pd.Series(data)
  counts = data.value_counts()
  data = counts
  colors = sns.color_palette('pastel')[0:10]
  plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')
  txt = f"Classes distribution of the {name} set"
  plt.figtext(0.5, 0.01, txt, wrap=True, horizontalalignment='center', fontsize=14)
  plt.savefig(f'my_plot_{name}.png')
  print("\n\n")
  plt.show()

plot_data(dataset_Y, "dataset")
plot_data(y_train, "train")
plot_data(y_test, "test")
print("\n\n")
print("Dataset histogram")
display(pd.DataFrame(dataset_Y).plot(kind="hist"))


## Machine Learning Approaches

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train, y_train, X_test, y_test, fit = True)

Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.42311191992720654
Precision:  0.43756242868268985
Recall:  0.42209765212709993
F1-micro:  0.42311191992720654
F1-macro:  0.39816958714619793


### Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

classifier = KNeighborsClassifier(n_neighbors=8)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 

Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.2747849705749208
Precision:  0.27326897510532777
Recall:  0.27529945739057854
F1-micro:  0.2747849705749208
F1-macro:  0.27084436530461636


In [ ]:
from datetime import datetime

def print_time():
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  print("Current Time =", current_time)

### MULTI LAYER PERCEPRON

In [ ]:
from sklearn.neural_network import MLPClassifier


print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(128, 64, 8), random_state=1, max_iter=200, learning_rate_init=0.001)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(128, 64, 8), random_state=1, max_iter=250, learning_rate_init=0.001)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(128, 64, 8), random_state=1, max_iter=250, learning_rate_init=0.0001)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 8), random_state=1, max_iter=250, learning_rate_init=0.0001)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(16, 8), random_state=1, max_iter=250, learning_rate_init=0.0001)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(128, 64, 8), random_state=1, max_iter=250, learning_rate_init=0.00001)  #OPTIMAL
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")

print_time()
classifier = MLPClassifier(hidden_layer_sizes=(128, 64, 8), random_state=1, max_iter=250, learning_rate_init=0.000001)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
print_time()

print("\n--------------\n")




--------------

Current Time = 20:30:55
Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.10185604345857854
Precision:  0.010190217391304348
Recall:  0.1
F1-micro:  0.10185604345857854
F1-macro:  0.018495684340320593
Current Time = 20:30:58

--------------

Current Time = 20:30:58
Fitting the model...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model fitted. Prediting on validation set...
Accuracy:  0.10185604345857854
Precision:  0.010190217391304348
Recall:  0.1
F1-micro:  0.10185604345857854
F1-macro:  0.018495684340320593
Current Time = 20:31:01

--------------

Current Time = 20:31:01
Fitting the model...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model fitted. Prediting on validation set...
Accuracy:  0.10502489814395655
Precision:  0.1768670309653916
Recall:  0.103134991814364
F1-micro:  0.10502489814395655
F1-macro:  0.02550697181474707
Current Time = 20:31:03

--------------

Current Time = 20:31:03
Fitting the model...


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:641: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (250) reached and the optimization hasn't converged yet.
  intercept_grads,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model fitted. Prediting on validation set...
Accuracy:  0.10321412403802625
Precision:  0.08522262607905498
Recall:  0.10139534883720931
F1-micro:  0.10321412403802625
F1-macro:  0.021288777964742683
Current Time = 20:31:11

--------------

Current Time = 20:31:11
Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.10638297872340426
Precision:  0.2602143182854537
Recall:  0.10450899197118335
F1-micro:  0.10638297872340424
F1-macro:  0.028229036404984094
Current Time = 20:31:24

--------------

Current Time = 20:31:24
Fitting the model...


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model fitted. Prediting on validation set...
Accuracy:  0.11226799456767768
Precision:  0.08091639172522237
Recall:  0.11401144343550884
F1-micro:  0.11226799456767768
F1-macro:  0.06584622048274702
Current Time = 20:31:29

--------------



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### SVC

In [ ]:
from sklearn.svm import LinearSVC

classifier = LinearSVC(max_iter=10000)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 

### RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import joblib as jb

"""
classifier_val = RandomForestClassifier(random_state=0)
print("Fine-tuning on validation set")
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier_val, 10, X_train_split=X_train, y_train_split=y_train, fit = True)
print("\n----------\n")
"""

classifier = RandomForestClassifier(random_state=0)
print("Testing on test set")
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True) 
# jb.dump(classifier, "/content/gdrive/MyDrive/Music Classification/random_forest")


Testing on test set
Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.8229968311453146
Precision:  0.8238986235818603
Recall:  0.8233676916597078
F1-micro:  0.8229968311453146
F1-macro:  0.8215068861881493


### ADA BOOST

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier(n_estimators=100, random_state=1)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True)

Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.4056133997283839
Precision:  0.39288508667814837
Recall:  0.4050988867392903
F1-micro:  0.4056133997283839
F1-macro:  0.36590077278004624


### GRADIENT BOOSTING

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier(n_estimators=150, random_state=0)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train, y_train, X_test, y_test, fit = True)

Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.8413709432817713
Precision:  0.8421760134964179
Recall:  0.8416580105337845
F1-micro:  0.8413709432817713
F1-macro:  0.8415255251505668


### DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(random_state=0)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train_split, y_train_split, X_validation, y_validation, fit = True)

Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.6011770031688547
Precision:  0.5996293011596643
Recall:  0.6015635048874498
F1-micro:  0.6011770031688547
F1-macro:  0.5995947197901336


### LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

#Failed to converge
# MAX_ITER 100, 1000, 2000, 10000

classifier = LogisticRegression(solver="sag", random_state=0, max_iter=10000)
accuracy, precision, recall, fmacro, fmicro = evaluate_classifier(classifier, 0, X_train, y_train, X_test, y_test, fit = True)

Fitting the model...
Model fitted. Prediting on validation set...
Accuracy:  0.35426144980285107
Precision:  0.32641808551651696
Recall:  0.35419208065500796
F1-micro:  0.3542614498028511
F1-macro:  0.3190230063649076


## DEEP LEARNING APPROCHES

### FFNN

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Flatten,Input,Dropout,BatchNormalization
from keras import models
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.callbacks import LearningRateScheduler,EarlyStopping
import matplotlib.pyplot as plt
import numpy as np

reg = regularizers.l2(0.001)
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# define the 784-256-128-10 architecture using Keras
model = models.Sequential(name="BatchNorm")

model.add(Input(shape=(58,)))
model.add(BatchNormalization())

#Dense                    50ep      100ep     200ep
#128 64 32
#128 64 32 10
#64 32 16
#64 32 32 16
#64 32 32     
#32 16 16                 .77
#32 16                    .78      .81        0.79           OPTIMAL
#32 16 10                 .76      .80

#model.add(Dense(64,activation='relu',kernel_regularizer=reg))
# model.add(Dense(64,activation='relu',kernel_regularizer=reg))
model.add(Dense(32,activation='relu',kernel_regularizer=reg))
model.add(Dense(16,activation='relu',kernel_regularizer=reg))
#model.add(Dense(10,activation='relu',kernel_regularizer=reg))


# model.add(Dropout(0.5)) 


model.add(Dense(10,activation='softmax',name='output'))
model.summary()

Model: "BatchNorm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_16 (Bat  (None, 58)               232       
 chNormalization)                                                
                                                                 
 dense_39 (Dense)            (None, 32)                1888      
                                                                 
 dense_40 (Dense)            (None, 16)                528       
                                                                 
 output (Dense)              (None, 10)                170       
                                                                 
Total params: 2,818
Trainable params: 2,702
Non-trainable params: 116
_________________________________________________________________


In [ ]:
import tensorflow as tf
from tensorflow import keras

# train the model using SGD
print("[INFO] training network...")

model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# X_train_split_ffnn = np.asarray(X_train).astype('float32')
# X_validation_ffnn = np.asarray(X_validation).astype('float32')

H = model.fit(X_train, y_train, validation_split=0.33, epochs=200, batch_size=32)

In [ ]:
H = model.fit(X_train, y_train, epochs=100, batch_size=32)

In [ ]:
import numpy as np
X_test_ffnn =  np.asarray(X_test).astype('float32')
model.evaluate(X_test, y_test)

104/104 [==============================] - 1s 8ms/step - loss: 0.7439 - accuracy: 0.8062


'\naccuracy, precision, recall, fmacro, fmicro = calculate_metrics(predicted, y_test)\n   \nprint("Accuracy: ", accuracy)\nprint("Precision: ", precision)\nprint("Recall: ", recall)\nprint("F1-micro: ", fmicro)\nprint("F1-macro: ", fmacro)'



```



```



---



```



```

# MFCC IMAGE CLASSIFICAITON

In [ ]:
import tensorflow as tf
print('tf.test.is_gpu_available:', tf.test.is_gpu_available())

tf.test.is_gpu_available: True


## Dataset Preprocessing

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import os
import math
import librosa
import joblib

DATASET_PATH = "/content/dataset/genres_original"
JSON_PATH = "./data_10.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
not_allowed = "/content/dataset/genres_original/jazz/jazz.00054.wav"

def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.

        :param dataset_path (str): Path to dataset
        :param json_path (str): Path to json file used to save MFCCs
        :param num_mfcc (int): Number of coefficients to extract
        :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
        :param hop_length (int): Sliding window for FFT. Measured in # of samples
        :param: num_segments (int): Number of segments we want to divide sample tracks into
        :return:
        """

    # dictionary to store mapping, labels, and MFCCs
    data = {
        "mapping": [],
        "labels": [],
        "mfcc": []
    }
    
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # loop through all genre sub-folder
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

        # ensure we're processing a genre sub-folder level
        if dirpath is not dataset_path:

            # save genre label (i.e., sub-folder name) in the mapping
            semantic_label = dirpath.split("/")[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))

            # process all audio files in genre sub-dir
            for f in filenames:
               
		# load audio file
                file_path = os.path.join(dirpath, f)
                if file_path != not_allowed :
                    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # process all segments of audio file
                    for d in range(num_segments):

                    # calculate start and finish sample for current segment
                        start = samples_per_segment * d
                        finish = start + samples_per_segment

                    # extract mfcc
                        mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                        mfcc = mfcc.T

                    # store only mfcc feature with expected number of vectors
                        if len(mfcc) == num_mfcc_vectors_per_segment:
                            data["mfcc"].append(mfcc.tolist())
                            data["labels"].append(i-1)
                            print("{}, segment:{}".format(file_path, d+1))

    # save MFCCs to json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4) 
                
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)


In [ ]:
import json
import numpy as np
DATASET_PATH = "/content/gdrive/MyDrive/Music Classification/data_10.json"
def load_data(dataset_path):
    with open(dataset_path,"r") as fp:
        data = json.load(fp)
    inputs = np.array(data["mfcc"])  
    targets = np.array(data["labels"])   
    
    return inputs , targets

inputs,targets = load_data(DATASET_PATH)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs, targets,test_size=0.33, random_state = 0)
X_train_split, X_validation, y_train_split, y_validation = train_test_split(X_train, y_train, test_size=0.3, random_state = 0)

In [ ]:
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print("\n")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")
print("\n")
print(f"X_train_split: {X_train_split.shape}")
print(f"y_train_split: {y_train_split.shape}")
print("\n")
print(f"X_validation: {X_validation.shape}")
print(f"y_validation: {y_validation.shape}")

## (Deep) Machine Learning Approches

### FFNN

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout

def create_model_FFNN():
  model = keras.Sequential([
      Input(shape=(130,13,)),
      Flatten(),
      # Dense(512,activation="relu"),
      # Dense(256,activation="relu"),
      Dense(128,activation="relu"),
      Dropout(0.4),
      Dense(64,activation="relu"),
      Dense(32, activation="relu"),
      Dense(16, activation="relu"),
      Dense(10,activation="softmax"),
  ])

  #64 32 16 10        .42
  #128 64 32 16 10        .42

  optimizer = keras.optimizers.Adam(learning_rate = 0.0001)
  model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
  model.summary()
  return model

In [ ]:
model_tuning = create_model_FFNN()
history_fnn = model_tuning.fit(X_train_split, y_train_split, validation_data=(X_validation, y_validation), epochs=50, batch_size=32)

In [ ]:
final_model = create_model_FFNN()
history_final = final_model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
final_model.evaluate(X_test, y_test)

94/94 [==============================] - 0s 3ms/step - loss: 1.7207 - accuracy: 0.3862


[1.720725417137146, 0.3861815631389618]

### CNN

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import regularizers

def build_model():
    model = keras.Sequential([
        Input(shape=((130, 13, 1,))),
        BatchNormalization(),

        Conv2D(8 , kernel_size=(3,3), activation = 'relu', kernel_regularizer=regularizers.l2(l=0.01), kernel_initializer='he_normal'),
        MaxPooling2D((2,2),strides=(2,2),padding='same'),
        
        #Dropout(0.3),
        
        Conv2D(32 , kernel_size=(3,3), activation = 'relu', kernel_regularizer=regularizers.l2(l=0.01), kernel_initializer='he_normal'),
        MaxPooling2D((2,2),strides=(2,2),padding='same'),
        #BatchNormalization(),
        # Dropout(0.3),
        #BatchNormalization(),   

        #Conv2D(16 , kernel_size=(3,3), activation = 'relu', kernel_regularizer=regularizers.l2(l=0.01), kernel_initializer='he_normal'),
        #MaxPooling2D((2,2),strides=(2,2),padding='same'),
        #BatchNormalization(),

        Flatten(),
        Dense(128, activation = 'relu'),
        Dense(64, activation = 'relu'),
        # Dense(32, activation = 'relu'),
        #Dropout(0.3),
        Dense(10, activation='softmax')


    ])

    #model.add(keras.layers.Conv2D(64 , (3,3) ,activation = 'relu'))
    #model.add(keras.layers.MaxPooling2D((3,3),strides=(2,2),padding='same'))
    #model.add(keras.layers.BatchNormalization())
    
    optimizer_cnn =  keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer = optimizer_cnn, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
from sklearn.model_selection import train_test_split

DATASET_PATH = "/content/gdrive/MyDrive/Music Classification/data_10.json"

def prepare_datasets(X_train, X_test):
    
    X_train = X_train[..., np.newaxis] # (num_samples,130,13,1)
    X_test = X_test[..., np.newaxis]
    
    return X_train, X_test

In [ ]:
X_train_cnn, X_test_cnn = prepare_datasets(X_train, X_test)
X_train_split_cnn, X_validation_cnn = prepare_datasets(X_train_split, X_validation)

In [ ]:
import joblib as jb

jb.dump(X_test_cnn, "/content/gdrive/MyDrive/Music Classification/X_test_cnn")
jb.dump(y_test, "/content/gdrive/MyDrive/Music Classification/y_test_cnn")


['/content/gdrive/MyDrive/Music Classification/y_test_cnn']

In [ ]:
print(f"X_train: {X_train_cnn.shape}")
print(f"y_train: {y_train.shape}")
print("\n")
print(f"X_test: {X_test_cnn.shape}")
print(f"y_test: {y_test.shape}")
print("\n")

X_train: (6690, 130, 13, 1)
y_train: (6690,)


X_test: (3296, 130, 13, 1)
y_test: (3296,)




In [ ]:
model_cnn = build_model()

model_cnn.fit(X_train_split_cnn, 
              y_train_split, 
              validation_data=(X_validation_cnn, y_validation), 
              batch_size=32,
              epochs=50)

In [ ]:
model_cnn = build_model()

model_cnn.fit(X_train_cnn, 
              y_train, 
              batch_size=32,
              epochs=50)

In [ ]:
model_cnn.evaluate(X_test_cnn, y_test)

94/94 [==============================] - 1s 4ms/step - loss: 1.2605 - accuracy: 0.6702


[1.2605401277542114, 0.6702269911766052]

In [ ]:
import joblib as jb

jb.dump(model_cnn, "/content/gdrive/MyDrive/Music Classification/model_cnn")

['/content/gdrive/MyDrive/Music Classification/model_cnn']

### RNN

In [ ]:
from sklearn.model_selection import train_test_split

X_train_rnn =  jb.load("/content/gdrive/MyDrive/Music Classification/rnn_X_train")
y_train_rnn = jb.load("/content/gdrive/MyDrive/Music Classification/rnn_y_train")
X_test_rnn = jb.load("/content/gdrive/MyDrive/Music Classification/rnn_X_test")
y_test_rnn = jb.load("/content/gdrive/MyDrive/Music Classification/rnn_y_test")

X_train_split, X_validation, y_train_split, y_validation = train_test_split(X_train_rnn, y_train_rnn, test_size=0.33, random_state=0)

In [ ]:
from tensorflow import keras

def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(130, 13,)))
    model.add(keras.layers.LSTM(64,return_sequences=True))
    model.add(keras.layers.LSTM(64))
    model.add(keras.layers.Dense(64,activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(10,activation='softmax'))
    model.summary()

    return model

In [ ]:
model_rnn = build_model()
optimizer =  keras.optimizers.Adam(learning_rate=0.001)
model_rnn.compile(optimizer = optimizer ,loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 130, 64)           19968     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_28 (Dense)            (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_29 (Dense)            (None, 10)                650       
                                                                 
Total params: 57,802
Trainable params: 57,802
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_rnn.fit(X_train_rnn, y_train_rnn, validation_data = (X_validation, y_validation), batch_size=32, epochs=30)

Epoch 1/30
210/210 [==============================] - 8s 39ms/step - loss: 1.4575 - accuracy: 0.4726 - val_loss: 1.3195 - val_accuracy: 0.5274
Epoch 2/30
210/210 [==============================] - 6s 27ms/step - loss: 1.3330 - accuracy: 0.5198 - val_loss: 1.2239 - val_accuracy: 0.5650
Epoch 3/30
210/210 [==============================] - 6s 27ms/step - loss: 1.2299 - accuracy: 0.5676 - val_loss: 1.1509 - val_accuracy: 0.5835
Epoch 4/30
210/210 [==============================] - 5s 22ms/step - loss: 1.2033 - accuracy: 0.5748 - val_loss: 1.0724 - val_accuracy: 0.6139
Epoch 5/30
210/210 [==============================] - 5s 22ms/step - loss: 1.1482 - accuracy: 0.5969 - val_loss: 1.1148 - val_accuracy: 0.6030
Epoch 6/30
210/210 [==============================] - 5s 22ms/step - loss: 1.0895 - accuracy: 0.6184 - val_loss: 0.9628 - val_accuracy: 0.6627
Epoch 7/30
210/210 [==============================] - 5s 23ms/step - loss: 1.0023 - accuracy: 0.6546 - val_loss: 0.8840 - val_accuracy: 0.7048

In [ ]:
model_rnn.evaluate(X_test_rnn, y_test_rnn)

103/103 [==============================] - 3s 13ms/step - loss: 0.7356 - accuracy: 0.7815


[0.7355664968490601, 0.7814871072769165]

# ENSEMBLE LEARNING
CNN + RANDOM FOREST

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import joblib as jb

X_test = jb.load("/content/gdrive/MyDrive/Music Classification/X_test")
y_test = jb.load("/content/gdrive/MyDrive/Music Classification/y_test")

file_name_test = [filename for filename in X_test["filename"]]
index_test = [dataset.index[dataset["filename"] == file_name][0] for file_name in file_name_test]
index_test = [index for index in index_test if index <= 9986]

targets_cnn = jb.load("/content/gdrive/MyDrive/Music Classification/targets_cnn")
inputs_cnn = jb.load("/content/gdrive/MyDrive/Music Classification/inputs_cnn")

# model_cnn_dump = jb.load("/content/gdrive/MyDrive/Music Classification/model_cnn")
random_forest_dump = jb.load("/content/gdrive/MyDrive/Music Classification/random_forest")

# jb.load("/content/gdrive/MyDrive/Music Classification/rnn_X_train")
# jb.load("/content/gdrive/MyDrive/Music Classification/rnn_y_train")
X_test_rnn = jb.load("/content/gdrive/MyDrive/Music Classification/rnn_X_test")
y_test_rnn = jb.load("/content/gdrive/MyDrive/Music Classification/rnn_y_test")
model_rnn_dump = jb.load("/content/gdrive/MyDrive/Music Classification/rnn_model")



In [ ]:
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")
print(f"Dataset cnn: {inputs_cnn.shape}")
print(f"Target cnn: {targets_cnn.shape}")

X_test: (3297, 59)
y_test: (3297,)
Dataset cnn: (9986, 130, 13, 1)
Target cnn: (9986, 1)


In [ ]:
import warnings

def ensamble_classifier_predicition(dataset, inputs_cnn, targets_cnn, model_dl, model_rf):
  counter = 1
  rigth_prediction = 0

  for i in index_test:
    X_current = dataset.iloc[i].drop(["filename", "label"], axis=0)
    y_current = dataset.iloc[i]["label"]
    X_cnn_current = inputs_cnn[i]
    y_cnn_current = targets_cnn[i][0]

    #print(f"Expected value {y_current} \n\n")
    print(f">> {counter}/{len(index_test)}")
    
    input = np.array(X_current).reshape(1, -1)

    probs_kf = _evalute_model_rf(model_rf, input)

    input =  np.array([X_cnn_current,])
    probs_dl = _evalute_model_dl(model_dl, input)

    mean_prob = np.mean( np.array([ probs_kf, probs_dl ]), axis=0 )
    pred = get_prediction(mean_prob)

    counter += 1

    
    if(pred == y_current):
      rigth_prediction += 1
    
  return rigth_prediction



def _evalute_model_rf(model, input):
  
  probs = random_forest_dump.predict_proba(input)
  #print("RandomForet")
  #_print_probs(probs)
  return probs


def _evalute_model_dl(model, input):
  probs = model.predict(input)
  #print("CNN")
  #_print_probs(probs)
  return probs


def get_prediction(mean_prob):
  #for prob in mean_prob[0]:
  #  print(f'{prob:.20f}\n\n')

  hig_prob = np.amax(mean_prob)
  #print(f"Highest probability: {hig_prob}")
  max_index = np.where(mean_prob[0] == hig_prob)
  #print(f"\n\nMax index: {max_index[0][0]}")
  return max_index[0][0]




warnings.filterwarnings('ignore')
right_prediction = ensamble_classifier_predicition(dataset, inputs_cnn, targets_cnn, model_rnn_dump, random_forest_dump)
print("\n\n--------------------------------------\n\n")
print(right_prediction)
print(f"Accuracy: {right_prediction/len(index_test)}")